# Data preparation 

_Akin Kazakci, Mines ParisTech - PSL University_

Input:
- (Facebook movement and population density dataset folders, French city codes (cities.csv)

Output:
- (movement_with_density_code_insee_'date'.csv)

We will have a look at the data and build a data structure where for each 'administrative zone' and 'time slice' (3x 8 hours per day), we will have 'movement (influx)' based on FB movement dataset and 'population density' (based on Facebook population density dataset). Facebook data does not give the real population density nor movement quantities but _indices_ (see later). 

In [1]:
import pandas as pd
from os import listdir

In [2]:
# %load src/clean_FB_file_names.py
import pandas as pd
from os import listdir

path = 'administrative_movements/'
filenames = listdir(path)
filenames


def clean_FB_file_names(path, string_to_remove,prefix):
    import os
    for count, filename in enumerate(os.listdir(path)): 
        dst = str(filename).replace(string_to_remove,prefix)
        src = path + filename 
        dst = path+ dst 

        os.rename(src, dst) 


if __name__ == "__main__":
    
    #change args 
    
    string_to_remove = 'France Coronavirus Disease Prevention Map Mar 05 2020 Id  Movement between Administrative Regions_'
    prefix = 'm_'
    path = 'administrative_movements/' #or 'administrative_density/

    clean_FB_file_names(path, string_to_remove,prefix)

Here is the information given by Facebook:

DataSet Id: 488834255146400

What specific pairs of places (admin regions) are people moving between more or less often than we would expect based on pre-crisis levels?

Movement maps illustrate patterns of movement of groups of people between different neighborhoods or cities over a period of several hours. By understanding these patterns, response organizations can better predict where resources will be needed, gain insight into patterns of evacuation, or predict where traffic will be most congested.

The following metrics are available:
* Date Time - The time period represented by the current map layer.
* Starting Location - The region where the movement of the group started.
* Ending Location: The region where the movement of the group ended.
* Length (km) - The distance traveled in kilometers.
* Baseline: People Moving - The total number of people who moved from Starting Location to Ending Location on average during the weeks before the disaster began.
* Crisis: People Moving - The total number of people who moved from Starting Location to Ending Location during the time period specified
* Difference - The difference between the number of people moving from Starting Location to Ending Location during the disaster compared to before the disaster.
* Percent Change - The percentage difference between the number of people moving from Starting Location to Ending Location during the disaster compared to before the disaster.
* Standard (Z) Score: The number of standard deviations by which the count of people moving during the crisis differs from the number of people moving during the baseline. Any z-value greater than 4 or smaller than -4 is clipped at 4 or -4.

I will start with movement data. Once it is loaded into a single dataframe, we need to add zip code for departments ('Code Insee') using an external file (cities.csv). This is a list of administrative zones in France with their map information (polygons) and various caracteristics.

Before merging movement data with cities codes, we need to clean files we downloaded and build a data structure.

In [3]:
path = 'administrative_movements/'
filenames = listdir(path)

#Function to read files into a dataframe 
def build_frames(f, path):
    path = path+f
    df = pd.DataFrame()
    if f.endswith('.csv'):
        df = pd.read_csv(path)
        #print (path)
    return df

In [4]:
# build a list of dataframes
frames = [ build_frames(f, path) for f in filenames ]

# concat dataframes (we can sort data later cause all dfs are timestamped)
movement_data = pd.concat(frames)

Later we will regroup data per day, instead of 8 hours slices. Thus, we add a date column.

In [5]:
# add a date column, 
movement_data['date'] = [row.date() for row in pd.to_datetime(movement_data.date_time)]

In [6]:
#for some reason FB data contains rows from great britain, delete those
movement_data = movement_data[movement_data['country'] == 'FR']

In [7]:
# there are 16818 rows where start and end polygons are the same; I dropped those lines
movement_data = movement_data[movement_data['start_polygon_id'] != movement_data['end_polygon_id']]

In [8]:
movement_data.shape

(51027, 21)

In [9]:
movement_data.columns

Index(['geometry', 'date_time', 'start_polygon_id', 'start_polygon_name',
       'end_polygon_id', 'end_polygon_name', 'length_km', 'tile_size',
       'country', 'level', 'n_crisis', 'n_baseline', 'n_difference',
       'percent_change', 'is_statistically_significant', 'z_score',
       'start_lat', 'start_lon', 'end_lat', 'end_lon', 'date'],
      dtype='object')

Let us load city codes and data. Obtaining this file required some work since Facebook polygons and France map does not always intersect.

In [10]:
city_codes = pd.read_csv('cities.csv', sep=';')


In the movement dataframe, we have starting and end polygons. So we need to add both departmental codes (two new columns).

In [11]:
# FIRST EXECUTE THIS
city_codes.rename(columns = {'geometry':'end_point', 'code_insee':'end_code_insee','polygon_id':'end_polygon_id'}, inplace = True) #',
city_codes = city_codes[['end_code_insee','end_polygon_id','end_point']]
city_codes.columns

Index(['end_code_insee', 'end_polygon_id', 'end_point'], dtype='object')

In [12]:
#movement_data.rename(columns = {'start_polygon_id':'polygon_id'}, inplace = True)
merge_attempt = pd.merge(movement_data, city_codes, how ='left', on='end_polygon_id')
merge_attempt#[movement_data['start_polygon_id']=='153026']

,geometry,date_time,start_polygon_id,start_polygon_name,end_polygon_id,end_polygon_name,length_km,tile_size,country,level,...,percent_change,is_statistically_significant,z_score,start_lat,start_lon,end_lat,end_lon,date,end_code_insee,end_point
0,"LINESTRING (6.4072265625 47.54678262206, 6.372...",2020-04-13 1600,151058,Haute-Saône,151057,Doubs,14.779808,12,FR,LEVEL3,...,-90.985577,0,-4.000000,47.641297,6.087378,47.165940,6.362767,2020-04-13,25,POINT (6.3627673495422 47.1659398827)
1,"LINESTRING (-2.00390625 48.60962767452532, -2....",2020-04-13 1600,162813,Ille-et-Vilaine,162816,Côtes-d'Armor,7.262711,12,FR,LEVEL3,...,-96.890547,0,-4.000000,48.155998,-1.638757,48.441082,-2.863818,2020-04-13,22,POINT (-2.8638179631241 48.441081670621)
2,LINESTRING (6.702952665441177 48.4702643598611...,2020-04-13 1600,138785,Meurthe-et-Moselle,141764,Vosges,13.670990,12,FR,LEVEL3,...,-94.109494,0,-4.000000,48.788773,6.161936,48.196375,6.380097,2020-04-13,88,POINT (6.380097025155799 48.196374688122)
3,"LINESTRING (6.7998046875 48.308484700771565, 6...",2020-04-13 1600,141764,Vosges,138785,Meurthe-et-Moselle,20.333646,12,FR,LEVEL3,...,-91.575818,0,-4.000000,48.196375,6.380097,48.788773,6.161936,2020-04-13,54,POINT (6.161935534586201 48.788772923342)
4,LINESTRING (6.801060267857143 43.5682258217796...,2020-04-13 1600,132360,Var,132356,Alpes-Maritimes,11.988855,12,FR,LEVEL3,...,-87.811956,0,-4.000000,43.441394,6.244436,43.937499,7.116540,2020-04-13,06,POINT (7.1165396780881 43.937498635409)
5,LINESTRING (-2.07916259765625 48.4890987554892...,2020-04-13 1600,162816,Côtes-d'Armor,162813,Ille-et-Vilaine,6.455673,12,FR,LEVEL3,...,-86.688852,0,-4.000000,48.441082,-2.863818,48.155998,-1.638757,2020-04-13,35,POINT (-1.6387567734127 48.155998402337)
6,"LINESTRING (6.845703125 47.52377930909915, 6.8...",2020-04-13 1600,151057,Doubs,153025,Territoire de Belfort,12.739222,12,FR,LEVEL3,...,-96.736292,0,-4.000000,47.165940,6.362767,47.631549,6.928786,2020-04-13,90,POINT (6.928786304666 47.63154940893)
7,"LINESTRING (6.903076171875 47.61107882306799, ...",2020-04-13 1600,153025,Territoire de Belfort,151057,Doubs,11.385253,12,FR,LEVEL3,...,-96.460449,0,-4.000000,47.631549,6.928786,47.165940,6.362767,2020-04-13,25,POINT (6.3627673495422 47.1659398827)
8,LINESTRING (6.9313742897727275 43.612189988350...,2020-04-13 1600,132356,Alpes-Maritimes,132360,Var,12.412757,12,FR,LEVEL3,...,-95.963796,0,-4.000000,43.937499,7.116540,43.441394,6.244436,2020-04-13,83,POINT (6.2444358836793 43.441394031236)
9,"LINESTRING (7.1630859375 48.99334514816719, 7....",2020-04-13 1600,141765,Moselle,138786,Bas-Rhin,1.552127,12,FR,LEVEL3,...,-92.027335,0,-4.000000,49.037552,6.661347,48.671473,7.551988,2020-04-13,67,POINT (7.551988030043201 48.67147272631099)


In [13]:
# THEN EXECUTE THIS
city_codes = pd.read_csv('cities.csv', sep=';')


city_codes.rename(columns = {'geometry':'start_point', 'code_insee':'start_code_insee','polygon_id':'start_polygon_id'}, inplace = True) #',
city_codes = city_codes[['start_code_insee','start_polygon_id','start_point']]
city_codes.columns

Index(['start_code_insee', 'start_polygon_id', 'start_point'], dtype='object')

In [14]:
merge_attempt = pd.merge(merge_attempt, city_codes, how ='left', on='start_polygon_id')

What we have now is a dataframe with start and end polygons and movement related quantities. For each start and end polygon we have added the French departmental codes ('code_insee') through merge with city_codes.

In [15]:
merge_attempt.columns

Index(['geometry', 'date_time', 'start_polygon_id', 'start_polygon_name',
       'end_polygon_id', 'end_polygon_name', 'length_km', 'tile_size',
       'country', 'level', 'n_crisis', 'n_baseline', 'n_difference',
       'percent_change', 'is_statistically_significant', 'z_score',
       'start_lat', 'start_lon', 'end_lat', 'end_lon', 'date',
       'end_code_insee', 'end_point', 'start_code_insee', 'start_point'],
      dtype='object')

In [16]:
#merge_attempt.drop(['lieu'], axis= 1, inplace = True)#,'nuts3','wikipedia','surf_km2', 'feature_id']
#merge_attempt.rename(columns = {'geometry_x':'polygon_geometry','geometry_y':'insee_point'}, inplace = True) 
#merge_attempt.rename(columns = {'insee_point':'end_insee_point'}, inplace = True) 

In [17]:
print (merge_attempt.shape)
print (merge_attempt.date_time.unique().shape) # We have three slices for each day (00:00-8:00, 8:00-16:00, 16:00-24:00)
print ('Number of unique origins',merge_attempt.start_polygon_name.unique().shape)
print ('Number of unique destinations', merge_attempt.end_polygon_name.unique().shape)
print ('Number of days', merge_attempt.date.unique().shape)

(51027, 25)
(186,)
Number of unique origins (94,)
Number of unique destinations (94,)
Number of days (62,)


In [18]:
# merge is successful, let us continue with movement data.
movement_data = merge_attempt

In [19]:
movement_data.to_csv('movements_with_insee_07-05-2020.csv')

In [20]:
print (movement_data.shape)
print (movement_data.date_time.unique().shape) # We have three slices for each day (00:00-8:00, 8:00-16:00, 16:00-24:00)
print ('Number of unique origins',movement_data.start_polygon_name.unique().shape)
print ('Number of unique destinations', movement_data.end_polygon_name.unique().shape)
print ('Number of days', movement_data.date.unique().shape)

(51027, 25)
(186,)
Number of unique origins (94,)
Number of unique destinations (94,)
Number of days (62,)


In [21]:
movement_data.head()

,geometry,date_time,start_polygon_id,start_polygon_name,end_polygon_id,end_polygon_name,length_km,tile_size,country,level,...,z_score,start_lat,start_lon,end_lat,end_lon,date,end_code_insee,end_point,start_code_insee,start_point
0,"LINESTRING (6.4072265625 47.54678262206, 6.372...",2020-04-13 1600,151058,Haute-Saône,151057,Doubs,14.779808,12,FR,LEVEL3,...,-4.0,47.641297,6.087378,47.165940,6.362767,2020-04-13,25,POINT (6.3627673495422 47.1659398827),70,POINT (6.0873779037648 47.641297246245)
1,"LINESTRING (-2.00390625 48.60962767452532, -2....",2020-04-13 1600,162813,Ille-et-Vilaine,162816,Côtes-d'Armor,7.262711,12,FR,LEVEL3,...,-4.0,48.155998,-1.638757,48.441082,-2.863818,2020-04-13,22,POINT (-2.8638179631241 48.441081670621),35,POINT (-1.6387567734127 48.155998402337)
2,LINESTRING (6.702952665441177 48.4702643598611...,2020-04-13 1600,138785,Meurthe-et-Moselle,141764,Vosges,13.670990,12,FR,LEVEL3,...,-4.0,48.788773,6.161936,48.196375,6.380097,2020-04-13,88,POINT (6.380097025155799 48.196374688122),54,POINT (6.161935534586201 48.788772923342)
3,"LINESTRING (6.7998046875 48.308484700771565, 6...",2020-04-13 1600,141764,Vosges,138785,Meurthe-et-Moselle,20.333646,12,FR,LEVEL3,...,-4.0,48.196375,6.380097,48.788773,6.161936,2020-04-13,54,POINT (6.161935534586201 48.788772923342),88,POINT (6.380097025155799 48.196374688122)
4,LINESTRING (6.801060267857143 43.5682258217796...,2020-04-13 1600,132360,Var,132356,Alpes-Maritimes,11.988855,12,FR,LEVEL3,...,-4.0,43.441394,6.244436,43.937499,7.116540,2020-04-13,06,POINT (7.1165396780881 43.937498635409),83,POINT (6.2444358836793 43.441394031236)


In [22]:
movement_data.columns

Index(['geometry', 'date_time', 'start_polygon_id', 'start_polygon_name',
       'end_polygon_id', 'end_polygon_name', 'length_km', 'tile_size',
       'country', 'level', 'n_crisis', 'n_baseline', 'n_difference',
       'percent_change', 'is_statistically_significant', 'z_score',
       'start_lat', 'start_lon', 'end_lat', 'end_lon', 'date',
       'end_code_insee', 'end_point', 'start_code_insee', 'start_point'],
      dtype='object')

In [23]:
(movement_data.shape[0] - movement_data[movement_data.end_code_insee == movement_data.start_code_insee].shape[0])/movement_data.shape[0]

0.9927097419013463

# Population density data 

Here are the explanations of the dataset and variables, as given by Facebook.
DataSet Id: 1246339958888741

Location density maps are heat maps, which show where people are located before, during and after a disaster and where populations have increased or decreased. We can compare this information to historical records, like population estimates based on satellite images. Comparing these data sets can help response organizations understand areas impacted by a natural disaster.

The polygon/vector version of the daily location map displays the following metrics aggregated by administrative boundaries:
* Date Time - The time period represented by the current map layer.
* Standard (Z) Score - The number of standard deviations by which the crisis population count in the location differs from the baseline count.
* Baseline:People - The average number of people we expect to be in the area during the specified time based on pre-disaster estimates.
* Crisis:People - The number of people observed in the tile during the selected time period.
* Difference - The difference between the population at the time of the crisis and the population during the baseline.
* Percent Change - The percentage difference between the population at the time of the crisis and the population during the baseline.

Let us check population density data and see what can be added to the previous data structure to enrich it.

In [24]:
path = 'administrative_density/'
filenames = listdir(path)

In [25]:
# build a list of dataframes
frames = [ build_frames(f, path) for f in filenames ]
# concat dataframes (we can sort data later cause all dfs are timestamped)
population_density_data = pd.concat(frames)

In [26]:
population_density_data['date'] = [row.date() for row in pd.to_datetime(population_density_data.date_time)]

In [27]:
population_density_data = population_density_data[population_density_data['country'] == 'FR']

In [28]:
population_density_data.columns

Index(['spaco_id', 'country', 'polygon_name', 'level', 'date_time',
       'n_baseline', 'n_crisis', 'density_baseline', 'density_crisis',
       'n_difference', 'percent_change', 'clipped_z_score', 'lat', 'lon',
       'date'],
      dtype='object')

In [29]:
population_density_data.polygon_name.unique().shape

(94,)

This dataset contains population density information. We will transfer some of this information to the movement_data we built above. We shall change columns names two times to be able to merge both on start and end polygons, so that density information for both start and end polygons of movement_data appears on that dataframe.

In [30]:
end_pop = population_density_data[['date_time', 'polygon_name','n_crisis','n_baseline','percent_change']]
end_pop.columns = ['date_time', 'end_polygon_name','end_density_crisis','end_density_baseline','end_density_percent_change']
end_pop.shape

(17484, 5)

In [31]:
start_pop = population_density_data[['date_time', 'polygon_name','n_crisis','n_baseline','percent_change']]
start_pop.columns = ['date_time', 'start_polygon_name','start_density_crisis','start_density_baseline','start_density_percent_change']
start_pop.shape

(17484, 5)

In [32]:
density_merge_attempt = pd.merge(movement_data,end_pop,how='left', on=['end_polygon_name','date_time'])

In [33]:
density_merge_attempt = pd.merge(density_merge_attempt,start_pop,how='left', on=['start_polygon_name','date_time'])

In [34]:
density_merge_attempt.shape

(51027, 31)

In [35]:
density_merge_attempt.columns

Index(['geometry', 'date_time', 'start_polygon_id', 'start_polygon_name',
       'end_polygon_id', 'end_polygon_name', 'length_km', 'tile_size',
       'country', 'level', 'n_crisis', 'n_baseline', 'n_difference',
       'percent_change', 'is_statistically_significant', 'z_score',
       'start_lat', 'start_lon', 'end_lat', 'end_lon', 'date',
       'end_code_insee', 'end_point', 'start_code_insee', 'start_point',
       'end_density_crisis', 'end_density_baseline',
       'end_density_percent_change', 'start_density_crisis',
       'start_density_baseline', 'start_density_percent_change'],
      dtype='object')

In [46]:
density_merge_attempt.to_csv('movement_with_density_code_insee_07-05-2020.csv')